## Modules

In [11]:
import os
import sys
sys.path.append('..')
from glob import glob

from time import time
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from bsoption.nyopchain import NYopchain
Opchain = NYopchain()

from litedata import holidaydictny, gettradedays
from litedata.loadyahoodata import getlitedata

### US stock list setting

In [8]:
from configparser import ConfigParser
configchain = ConfigParser()
configchain.read('setting_yahoochain.ini')
chainpath = configchain['paths'].get('chainpath')

configassets = configchain['assets']
etflist = list(configassets.get('etf').split(','))
chipslist = list(configassets.get('chip').split(','))
assetlist = etflist + chipslist

## Option chain query on one asset

### Example 1: single expiry, two adjacent trading days, only put, order by strike then tradedate

In [2]:
starttime = time()
asset1 = 'NVDA'

inputdict1 = {'asset': asset1,  
              'optype': ('P', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 150,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 180,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': '2022-08-27',  # query expiry startdate (inclusive)
              'endexpiry': '2022-09-02',  # query expiry enddate (inclusive)              
              'starttd': '2022-08-25',  # query period startdate (inclusive)
              'endtd': '2022-08-26'  # query period startdate (inclusive) 
             }

orderfield1 = ['optype', 'strike', 'tradedate']
dfop1 = Opchain.loadopdata(inputdict1, orderfield=orderfield1)

endtime = time()
duration = round(endtime - starttime, 4)
print(f'Time elapsed query on {asset1} next week option: {duration}s')

dfop1

Time elapsed query on NVDA next week option: 0.2209s


,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-08-25,NVDA,P,2022-09-02,150.0,62.89,3692.0,3178,0.17,0.15,0.18,0.165
1,2022-08-26,NVDA,P,2022-09-02,150.0,57.96,10752.0,3246,1.05,1.02,1.05,1.035
2,2022-08-25,NVDA,P,2022-09-02,152.5,61.13,685.0,1050,0.23,0.21,0.24,0.225
3,2022-08-26,NVDA,P,2022-09-02,152.5,57.30,3013.0,1337,1.48,1.47,1.49,1.480
4,2022-08-25,NVDA,P,2022-09-02,155.0,59.38,1897.0,1831,0.30,0.29,0.32,0.305
5,2022-08-26,NVDA,P,2022-09-02,155.0,56.57,5881.0,1954,2.07,2.05,2.07,2.060
6,2022-08-25,NVDA,P,2022-09-02,157.5,57.72,1360.0,1108,0.42,0.40,0.43,0.415
7,2022-08-26,NVDA,P,2022-09-02,157.5,55.71,4499.0,1299,2.77,2.77,2.81,2.790
8,2022-08-25,NVDA,P,2022-09-02,160.0,56.06,8675.0,3841,0.56,0.55,0.57,0.560
9,2022-08-26,NVDA,P,2022-09-02,160.0,54.88,26134.0,6545,3.73,3.65,3.75,3.700


### Single option contract across days

In [3]:
inputdict2 = {'asset': 'QQQ',  
              'optype': ('P', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 300,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 300,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': '2022-09-16',  # query expiry startdate (inclusive)
              'endexpiry': '2022-09-16',  # query expiry enddate (inclusive)              
              'starttd': '2022-08-01',  # query period startdate (inclusive)
              'endtd': '2022-09-16'  # query period startdate (inclusive) 
             }

dfop2 = Opchain.loadopdata(inputdict2)
dfop2

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-08-01,QQQ,P,2022-09-16,300.0,28.61,7870.0,125557,6.29,6.24,6.28,6.260
1,2022-08-02,QQQ,P,2022-09-16,300.0,30.13,12331.0,125399,7.12,6.98,7.03,7.005
2,2022-08-03,QQQ,P,2022-09-16,300.0,30.15,10849.0,122700,4.66,4.60,4.63,4.615
3,2022-08-04,QQQ,P,2022-09-16,300.0,30.24,6000.0,125882,4.19,4.18,4.22,4.200
4,2022-08-05,QQQ,P,2022-09-16,300.0,29.58,6172.0,128425,4.44,4.42,4.47,4.445
5,2022-08-08,QQQ,P,2022-09-16,300.0,29.61,12945.0,129045,4.34,4.32,4.36,4.340
6,2022-08-09,QQQ,P,2022-09-16,300.0,29.60,12082.0,132695,5.12,5.07,5.13,5.100
7,2022-08-10,QQQ,P,2022-09-16,300.0,29.27,12717.0,128812,2.95,2.94,2.98,2.960
8,2022-08-11,QQQ,P,2022-09-16,300.0,28.99,12519.0,129764,3.12,3.10,3.13,3.115
9,2022-08-12,QQQ,P,2022-09-16,300.0,29.60,9333.0,133910,2.17,2.15,2.17,2.160


In [4]:
asset3 = 'TSLA'
optype3 = 'P'
strike3 = 260
expiry3 = '2022-10-07'
starttd3 = '2022-09-11'
endtd3 = '2022-09-30'

inputdict3 = {'asset': 'TSLA',  'optype': (optype3, optype3), 
              'strike_lowerbound': strike3,  'strike_upperbound': strike3,  
              'startexpiry': expiry3,  'endexpiry': expiry3,            
              'starttd': starttd3, 'endtd': endtd3}

dfop3 = Opchain.loadopdata(inputdict3)
dfop3

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-09-12,TSLA,P,2022-10-07,260.0,56.42,761.0,1473,3.00,2.96,3.10,3.030
1,2022-09-13,TSLA,P,2022-10-07,260.0,58.41,477.0,1646,5.03,5.00,5.30,5.150
2,2022-09-14,TSLA,P,2022-10-07,260.0,60.36,579.0,1879,3.55,3.40,3.65,3.525
3,2022-09-15,TSLA,P,2022-10-07,260.0,60.36,808.0,2018,3.15,3.00,3.30,3.150
4,2022-09-16,TSLA,P,2022-10-07,260.0,60.86,411.0,2503,3.05,2.92,3.20,3.060
5,2022-09-19,TSLA,P,2022-10-07,260.0,61.94,704.0,2654,1.96,1.87,2.01,1.940
6,2022-09-20,TSLA,P,2022-10-07,260.0,64.77,665.0,2822,2.10,2.02,2.16,2.090
7,2022-09-21,TSLA,P,2022-10-07,260.0,62.94,1665.0,2964,2.50,2.44,2.55,2.495
8,2022-09-22,TSLA,P,2022-10-07,260.0,61.88,1551.0,2752,3.90,3.80,4.00,3.900
9,2022-09-23,TSLA,P,2022-10-07,260.0,63.70,8939.0,2941,7.00,6.90,7.15,7.025


### Last trading day's option chain with strike in a price range

In [7]:
today = (datetime.today() - timedelta(hours=6)).strftime('%Y-%m-%d')
tdlist = gettradedays(holidaydictny)
lasttd = [dtstr for dtstr in tdlist if dtstr < today][-1]
ltddate = datetime.strptime(lasttd, '%Y-%m-%d')
nextexpiry = (ltddate + timedelta(days=7)).strftime('%Y-%m-%d')
next2expiry = (ltddate + timedelta(days=13)).strftime('%Y-%m-%d')

inputdict3 = {'asset': 'NFLX',  
              'optype': ('C', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 270,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 330,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': nextexpiry,  # query expiry startdate (inclusive)
              'endexpiry': next2expiry,  # query expiry enddate (inclusive)              
              'starttd': lasttd,  # query period startdate (inclusive)
              'endtd': lasttd  # query period startdate (inclusive) 
             }

dfop3 = Opchain.loadopdata(inputdict3)
dfop3

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-11-03,NFLX,C,2022-11-11,270.0,53.80,414.0,293,8.12,8.00,8.20,8.100
1,2022-11-03,NFLX,C,2022-11-11,272.5,53.22,216.0,32,7.10,6.80,7.00,6.900
2,2022-11-03,NFLX,C,2022-11-11,275.0,53.19,892.0,482,5.93,5.80,6.00,5.900
3,2022-11-03,NFLX,C,2022-11-11,277.5,52.76,284.0,104,5.00,4.85,5.05,4.950
4,2022-11-03,NFLX,C,2022-11-11,280.0,52.61,455.0,433,4.25,4.05,4.25,4.150
5,2022-11-03,NFLX,C,2022-11-11,282.5,52.45,182.0,139,3.46,3.35,3.55,3.450
6,2022-11-03,NFLX,C,2022-11-11,285.0,52.43,1588.0,334,2.91,2.80,2.92,2.860
7,2022-11-03,NFLX,C,2022-11-11,287.5,52.30,351.0,260,2.42,2.29,2.39,2.340
8,2022-11-03,NFLX,C,2022-11-11,290.0,52.32,1574.0,715,2.00,1.86,1.97,1.915
9,2022-11-03,NFLX,C,2022-11-11,292.5,52.27,80.0,200,1.69,1.50,1.60,1.550


## Gamma Exposure (GEX)

### OHLC of all stocks

In [10]:
dfohlcall = getlitedata(assetlist)
asset0 = 'NVDA'
dfohlcall.loc['2022-10':, f'{asset0}_cl']

NameError: name 'getlitedata' is not defined